## synthesize.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch

/content/drive/MyDrive/AudioData_kss/kss-Korean-FastSpeech2-Pytorch


In [ ]:
!pip install g2pk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 26.9 MB/s eta 0:00:00


In [ ]:
!pip install scipy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install jamo

In [ ]:
!pip install librosa

In [ ]:
!pip install tensorboard

In [ ]:
!pip install librosa

In [ ]:
!pip install tgt

  Preparing metadata (setup.py) ... done
  Created wheel for tgt: filename=tgt-1.4.4-py3-none-any.whl size=28903 sha256=c5966aa4b7d2bd0927657516c0c6a981ff33d5c7260a7d7bd44211fa81573523
  Stored in directory: /root/.cache/pip/wheels/09/e6/aa/821531faeb4e05a65d1c763570e90791467cf0c3f1622dc7e2
Successfully built tgt


In [ ]:
import torch
print(torch.__version__)

2.1.0+cu118


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import hparams as hp
import os

In [ ]:
hp.synthesize_step =33000

In [ ]:
# hp.restore_step = 8900
# hp.learning_rate = 0.00001
# # hp.save_step = 100
# hp.synthesize_step = 187000

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import hparams as hp
import librosa
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=hp.synth_visible_devices

import argparse
import re
from string import punctuation

from fastspeech2 import FastSpeech2
from vocoder import vocgan_generator

from text import text_to_sequence, sequence_to_text
import utils
import audio as Audio

import codecs
from g2pk import G2p
from jamo import h2j

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

def kor_preprocess(text):
    g2p=G2p()
    text = re.sub(r'[,!“”‘’〈〉《》「」『』…´`]', '', text)
    text = re.sub(r'[·/]', ' ', text)
    phone = g2p(text)
    print('after g2p: ',phone)
    phone = h2j(phone)
    print('after h2j: ',phone)
    phone = list(filter(lambda p: p != 'ff ', phone))
    phone = '{' + '}{'.join(phone) + '}'
    print('phone: ',phone)
    phone = re.sub(r'\{[^\w\s]?·\}', '{sil}', phone)
    print('after re.sub: ',phone)
    phone = phone.replace('}{', ' ')

    print('|' + phone + '|')
    sequence = np.array(text_to_sequence(phone,hp.text_cleaners))
    sequence = np.stack([sequence])
    return torch.from_numpy(sequence).long().to(device)

def get_FastSpeech2(num):
    checkpoint_path = os.path.join(hp.checkpoint_path, "checkpoint_{}.pth.tar".format(num))
    model = nn.DataParallel(FastSpeech2())
    model.load_state_dict(torch.load(checkpoint_path)['model'])
    model.requires_grad = False
    model.eval()
    return model

def synthesize(model, vocoder, text, sentence, ref_path, dur_pitch_energy_aug, prefix=''):
    sentence = sentence[:10] # long filename will result in OS Error

    mean_mel, std_mel = torch.tensor(np.load(os.path.join(hp.preprocessed_path, "mel_stat.npy")), dtype=torch.float).to(device)
    mean_f0, std_f0 = f0_stat = torch.tensor(np.load(os.path.join(hp.preprocessed_path, "f0_stat.npy")), dtype=torch.float).to(device)
    mean_energy, std_energy = energy_stat = torch.tensor(np.load(os.path.join(hp.preprocessed_path, "energy_stat.npy")), dtype=torch.float).to(device)

    mean_mel, std_mel = mean_mel.reshape(1, -1), std_mel.reshape(1, -1)
    mean_f0, std_f0 = mean_f0.reshape(1, -1), std_f0.reshape(1, -1)
    mean_energy, std_energy = mean_energy.reshape(1, -1), std_energy.reshape(1, -1)

    src_len = torch.from_numpy(np.array([text.shape[1]])).to(device)

    wav, _ = librosa.load(ref_path)
    # wav = wav[int(hp.sampling_rate*start):int(hp.sampling_rate*end)]

    mel_spectrogram, _ = Audio.tools.get_mel_from_wav(torch.FloatTensor(wav))
    mel_spectrogram = torch.from_numpy(mel_spectrogram.numpy().astype(np.float32).T).unsqueeze(0)
    ref_mels = mel_spectrogram[:, 1:, :]

    mel, mel_postnet, log_duration_output, f0_output, energy_output, _, _, mel_len = model(text, src_len, ref_mels, dur_pitch_energy_aug=dur_pitch_energy_aug, f0_stat=f0_stat, energy_stat=energy_stat)

    mel_torch = mel.transpose(1, 2).detach()
    mel_postnet_torch = mel_postnet.transpose(1, 2).detach()
    f0_output = f0_output[0]
    energy_output = energy_output[0]

    mel_torch = utils.de_norm(mel_torch.transpose(1, 2), mean_mel, std_mel)
    mel_postnet_torch = utils.de_norm(mel_postnet_torch.transpose(1, 2), mean_mel, std_mel).transpose(1, 2)
    f0_output = utils.de_norm(f0_output, mean_f0, std_f0).squeeze().detach().cpu().numpy()
    energy_output = utils.de_norm(energy_output, mean_energy, std_energy).squeeze().detach().cpu().numpy()

    if not os.path.exists(hp.test_path):
        os.makedirs(hp.test_path)

    #Audio.tools.inv_mel_spec(mel_postnet_torch[0], os.path.join(hp.test_path, '{}_griffin_lim_{}.wav'.format(prefix, sentence)))

    if vocoder is not None:
        if hp.vocoder.lower() == "vocgan":
            utils.vocgan_infer(mel_postnet_torch, vocoder, path=os.path.join(hp.test_path, '{}_{}_{}.wav'.format(prefix, hp.vocoder, sentence)))

    utils.plot_data([(mel_postnet_torch[0].detach().cpu().numpy(), f0_output, energy_output)], ['Synthesized Spectrogram'], filename=os.path.join(hp.test_path, '{}_{}.png'.format(prefix, sentence)))


[nltk_data] Downloading package cmudict to /root/nltk_data...


True


[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
hp.synthesize_step

33000

In [ ]:
hp.checkpoint_path

'/content/drive/MyDrive/AudioData_kss/kss-Korean-FastSpeech2-Pytorch/ckpt/emo'

In [ ]:
ref_paths = ["/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/ref_wav/1_0020.wav",
             "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/ref_wav/0050_G2A4E2S0C3_HJH_001999.wav",
             "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/ref_wav/0050_G2A4E5S1C0_HJH_001995.wav"]

In [ ]:
# Test
# parser = argparse.ArgumentParser()
# parser.add_argument('--step', type=int, default=30000)
# args = parser.parse_args()

dur_pitch_energy_aug = [1.0, 1.0, 1.0]    	# [duration, pitch, energy]

model = get_FastSpeech2(hp.synthesize_step).to(device)
if hp.vocoder == 'vocgan':
    vocoder = utils.get_vocgan(ckpt_path=hp.vocoder_pretrained_model_path)
else:
    vocoder = None

# #kss
# eval_sentence=['그는 괜찮은 척하려고 애쓰는 것 같았다','그녀의 사랑을 얻기 위해 애썼지만 헛수고였다','용돈을 아껴써라','그는 아내를 많이 아낀다','요즘 공부가 안돼요','한 여자가 내 옆에 앉았다']
# train_sentence=['가까운 시일 내에 한번, 댁으로 찾아가겠습니다','우리의 승리는 기적에 가까웠다','아이들의 얼굴에는 행복한 미소가 가득했다','헬륨은 공기보다 가볍다','이것은 간단한 문제가 아니다']
# test_sentence=['안녕하세요, 한동대학교 딥러닝 연구실입니다.', '이 프로젝트가 여러분에게 도움이 되었으면 좋겠습니다.', '시간이 촉박해요','이런, 큰일 났어','좀 더 먹지 그래?','제가 뭘 잘못했죠?','더 이상 묻지마']

# g2p=G2p()
# print('which sentence do you want?')
# print('1.eval_sentence 2.train_sentence 3.test_sentence 4.create new sentence')

# mode=input()
# print('you went for mode {}'.format(mode))
# if mode=='4':
#     print('input sentence')
#     sentence = input()
# elif mode=='1':
#     sentence = eval_sentence
# elif mode=='2':
#     sentence = train_sentence
# elif mode=='3':
#     sentence = test_sentence
# else:
#   sentence = mode
#   mode = '4'



### Demo

In [ ]:
print('input sentence')
sentence = input()

input sentence
반갑습니다


In [ ]:
print('sentence that will be synthesized: ')
print(sentence)
file_names = []
for idx in range(len(ref_paths)):
    ref_path = ref_paths[idx]
    file_names.append('ref_idx_{}-step_{}-duration_{}-pitch_{}-energy_{}'.format(idx, hp.synthesize_step, dur_pitch_energy_aug[0], dur_pitch_energy_aug[1], dur_pitch_energy_aug[2]))
    # if mode != '4':
    #     for s in sentence:
    #         text = kor_preprocess(s)

    #         print("text", text)
    #         synthesize(model, vocoder, text, s, ref_path, dur_pitch_energy_aug, prefix=file_names[idx])
    # else:
    text = kor_preprocess(sentence)
    if text[0, -1] == 82 : text = text[:, :-1]
    synthesize(model, vocoder, text, sentence, ref_path, dur_pitch_energy_aug, prefix=file_names[idx])


sentence that will be synthesized: 
반갑습니다
after g2p:  반갑씀니다
after h2j:  반갑씀니다
phone:  {ᄇ}{ᅡ}{ᆫ}{ᄀ}{ᅡ}{ᆸ}{ᄊ}{ᅳ}{ᆷ}{ᄂ}{ᅵ}{ᄃ}{ᅡ}
after re.sub:  {ᄇ}{ᅡ}{ᆫ}{ᄀ}{ᅡ}{ᆸ}{ᄊ}{ᅳ}{ᆷ}{ᄂ}{ᅵ}{ᄃ}{ᅡ}
|{ᄇ ᅡ ᆫ ᄀ ᅡ ᆸ ᄊ ᅳ ᆷ ᄂ ᅵ ᄃ ᅡ}|
after g2p:  반갑씀니다
after h2j:  반갑씀니다
phone:  {ᄇ}{ᅡ}{ᆫ}{ᄀ}{ᅡ}{ᆸ}{ᄊ}{ᅳ}{ᆷ}{ᄂ}{ᅵ}{ᄃ}{ᅡ}
after re.sub:  {ᄇ}{ᅡ}{ᆫ}{ᄀ}{ᅡ}{ᆸ}{ᄊ}{ᅳ}{ᆷ}{ᄂ}{ᅵ}{ᄃ}{ᅡ}
|{ᄇ ᅡ ᆫ ᄀ ᅡ ᆸ ᄊ ᅳ ᆷ ᄂ ᅵ ᄃ ᅡ}|
after g2p:  반갑씀니다
after h2j:  반갑씀니다
phone:  {ᄇ}{ᅡ}{ᆫ}{ᄀ}{ᅡ}{ᆸ}{ᄊ}{ᅳ}{ᆷ}{ᄂ}{ᅵ}{ᄃ}{ᅡ}
after re.sub:  {ᄇ}{ᅡ}{ᆫ}{ᄀ}{ᅡ}{ᆸ}{ᄊ}{ᅳ}{ᆷ}{ᄂ}{ᅵ}{ᄃ}{ᅡ}
|{ᄇ ᅡ ᆫ ᄀ ᅡ ᆸ ᄊ ᅳ ᆷ ᄂ ᅵ ᄃ ᅡ}|


In [ ]:
import IPython.display as ipd
import os

In [ ]:
# index = 0
# print("reference audio", index)
# ipd.Audio(ref_paths[index])

In [ ]:
hp.test_path

'/content/drive/MyDrive/AudioData_kss/kss-Korean-FastSpeech2-Pytorch/results'

In [ ]:
index = 0
print("synthesized audio by reference audio", index)
ipd.Audio(os.path.join(hp.test_path, "{}_{}_{}.wav".format(file_names[index], hp.vocoder, sentence[:10])))

synthesized audio by reference audio 0


In [ ]:
# index = 1
# print("reference audio", index)
# ipd.Audio(ref_paths[index])

In [ ]:
index = 1
print("synthesized audio by reference audio", index)
ipd.Audio(os.path.join(hp.test_path, "{}_{}_{}.wav".format(file_names[index], hp.vocoder, sentence[:10])))

synthesized audio by reference audio 1


In [ ]:
# index = 2
# print("reference audio", index)
# ipd.Audio(ref_paths[index])

In [ ]:
index = 2
print("synthesized audio by reference audio", index)
ipd.Audio(os.path.join(hp.test_path, "{}_{}_{}.wav".format(file_names[index], hp.vocoder, sentence[:10])))

synthesized audio by reference audio 2
